In [1]:
!pip install streamlit langchain-community langchain-openai langchain-text-splitters requests pathlib pypdf

In [2]:
## Handle secrets either from env vars or streamlit manager
import streamlit as st
import os
api_key = os.getenv("LITELLM_KEY")
if api_key is None:
    api_key = st.secrets["LITELLM_KEY"]
cirrus_key = os.getenv("CIRRUS_KEY")
if cirrus_key is None:
    cirrus_key = st.secrets["CIRRUS_KEY"]        


In [3]:
import os
import requests
import zipfile

def download_and_unzip(url, output_dir):
    response = requests.get(url)
    zip_file_path = os.path.basename(url)
    with open(zip_file_path, 'wb') as f:
        f.write(response.content)
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(output_dir)
    os.remove(zip_file_path)

download_and_unzip("https://minio.carlboettiger.info/public-data/hwc.zip", "hwc")

In [4]:
import pathlib
from langchain_community.document_loaders import PyPDFLoader

def pdf_loader(path):
    all_documents = []
    docs_dir = pathlib.Path(path)
    for file in docs_dir.iterdir():
        loader = PyPDFLoader(file)
        documents = loader.load()
        all_documents.extend(documents)
    return all_documents

docs = pdf_loader('hwc/')


In [5]:

# NRP embedding model tends to throw errors
# embedding = OpenAIEmbeddings(model = "embed-mistral", api_key = api_key, base_url = "https://llm.nrp-nautilus.io")



In [6]:
## Use the model on Cirrus instead:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(
                 model = "cirrus",
                 api_key =  cirrus_key, 
                 base_url = "https://llm.cirrus.carlboettiger.info/v1",
)

In [7]:
# Build a retrival agent
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=500)
splits = text_splitter.split_documents(docs)


In [8]:
# slow part here, runs on remote GPU
from langchain_core.vectorstores import InMemoryVectorStore
vectorstore = InMemoryVectorStore.from_documents(documents = splits, embedding = embedding)
retriever = vectorstore.as_retriever()

In [9]:


# Choose any of the models listed by their short-name:
# see `curl -H "Authorization: Bearer $OPENAI_API_KEY" https://llm.nrp-nautilus.io/v1/models`
models = {"llama3": "llama3-sdsc", 
          "deepseek-small": "DeepSeek-R1-Distill-Qwen-32B",
          "deepseek": "deepseek-r1-qwen-qualcomm",
          "gemma3": "gemma3",
          "phi3": "phi3",
          "olmo": "olmo"
         }

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = models['gemma3'],
                 api_key = api_key, 
                 base_url = "https://llm.nrp-nautilus.io",  
                 temperature=0)


from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following scientific articles as the retrieved context to answer "
    "the question. Appropriately cite the articles from the context on which your answer is based using (Author, Year) format. "
    "Do not attempt to cite articles that are not in the context."
    "If you don't know the answer, say that you don't know."
    "Use up to five sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [10]:
prompt = "I live in Tanzania and am having issues with lions breaking into my boma and preying on cattle. What interventions might work best for me?"
results = rag_chain.invoke({"input": prompt})
results

{'input': 'I live in Tanzania and am having issues with lions breaking into my boma and preying on cattle. What interventions might work best for me?',
 'context': [Document(id='20f4ca4d-9fb3-42bd-b433-a4a0437bd86a', metadata={'producer': 'Acrobat Distiller 10.1.10 (Windows); modified using iText 4.2.0 by 1T3XT', 'creator': 'LaTeX with hyperref package', 'creationdate': '2020-01-16T12:33:42+05:30', 'keywords': '', 'moddate': '2025-05-27T12:12:25-07:00', 'subject': 'Conservation Biology 2020.34:232-243', 'wps-proclevel': '3', 'wps-journaldoi': '10.1111/(ISSN)1523-1739', 'author': '', 'title': 'Predicting intervention priorities for wildlife conflicts', 'wps-articledoi': '10.1111/cobi.13372', 'source': 'hwc/Baynham-Herd et al. 2019.pdf', 'total_pages': 12, 'page': 10, 'page_label': '242'}, page_content='242 Conﬂict Intervention Priorities\nhelps foster more effective collaboration (Game et al.\n2013; Lute et al. 2018). Third, both the survey results\nand feedback were consistent with rec

In [11]:
prompt = "What are the most cost-effective prevention methods for elephants raiding my crops?"

results = rag_chain.invoke({"input": prompt})
results

{'input': 'What are the most cost-effective prevention methods for elephants raiding my crops?',
 'context': [Document(id='20f4ca4d-9fb3-42bd-b433-a4a0437bd86a', metadata={'producer': 'Acrobat Distiller 10.1.10 (Windows); modified using iText 4.2.0 by 1T3XT', 'creator': 'LaTeX with hyperref package', 'creationdate': '2020-01-16T12:33:42+05:30', 'keywords': '', 'moddate': '2025-05-27T12:12:25-07:00', 'subject': 'Conservation Biology 2020.34:232-243', 'wps-proclevel': '3', 'wps-journaldoi': '10.1111/(ISSN)1523-1739', 'author': '', 'title': 'Predicting intervention priorities for wildlife conflicts', 'wps-articledoi': '10.1111/cobi.13372', 'source': 'hwc/Baynham-Herd et al. 2019.pdf', 'total_pages': 12, 'page': 10, 'page_label': '242'}, page_content='242 Conﬂict Intervention Priorities\nhelps foster more effective collaboration (Game et al.\n2013; Lute et al. 2018). Third, both the survey results\nand feedback were consistent with recent scholarship\n(Redpath et al. 2017) that highlights 

In [12]:
rag_chain.invoke({"input": 
                  "I have a small herd of goats and cattle and I am worried about jaguars preying on them. What preventative measures can I take?"
                 })

{'input': 'I have a small herd of goats and cattle and I am worried about jaguars preying on them. What preventative measures can I take?',
 'context': [Document(id='9e597163-975a-4020-a2d3-848859b86bce', metadata={'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with hyperref', 'creationdate': '2022-06-23T12:24:35+08:00', 'author': 'Lydia N. Tiller, Ernest Oniba, Godfrey Opira, Ewan J. Brennan, Lucy E. King, Victor Ndombi, Derick Wanjala and Marion R. Robertson', 'keywords': 'African elephants; human-elephant conflict; crop raiding; olfactory mitigation;elephant repellent', 'moddate': '2022-06-23T06:36:53+02:00', 'subject': 'Human–elephant conflict is increasing across many parts of Asia and Africa. Mitigating elephant crop raiding has become a major focus of conservation intervention, however, many existing methods for tackling this problem are expensive and difficult to execute. Thus, there is a need for more affordable, farm-based methods. Testing these methods is key to ensuring th

In [13]:
rag_chain.invoke({"input": "I am trying to prevent coyotes from eating the calves of my free-range cattle. What may work best?"})

{'input': 'I am trying to prevent coyotes from eating the calves of my free-range cattle. What may work best?',
 'context': [Document(id='20f4ca4d-9fb3-42bd-b433-a4a0437bd86a', metadata={'producer': 'Acrobat Distiller 10.1.10 (Windows); modified using iText 4.2.0 by 1T3XT', 'creator': 'LaTeX with hyperref package', 'creationdate': '2020-01-16T12:33:42+05:30', 'keywords': '', 'moddate': '2025-05-27T12:12:25-07:00', 'subject': 'Conservation Biology 2020.34:232-243', 'wps-proclevel': '3', 'wps-journaldoi': '10.1111/(ISSN)1523-1739', 'author': '', 'title': 'Predicting intervention priorities for wildlife conflicts', 'wps-articledoi': '10.1111/cobi.13372', 'source': 'hwc/Baynham-Herd et al. 2019.pdf', 'total_pages': 12, 'page': 10, 'page_label': '242'}, page_content='242 Conﬂict Intervention Priorities\nhelps foster more effective collaboration (Game et al.\n2013; Lute et al. 2018). Third, both the survey results\nand feedback were consistent with recent scholarship\n(Redpath et al. 2017) t

In [14]:
rag_chain.invoke({"input": "We have major issues with deer raiding our large agricultural fields. Is there anything I can try to prevent this that won’t break the bank?"})

{'input': 'We have major issues with deer raiding our large agricultural fields. Is there anything I can try to prevent this that won’t break the bank?',
 'context': [Document(id='7d64f897-b500-4aac-8f4c-cf0bd19c062e', metadata={'producer': 'PyPDF', 'creator': 'Elsevier', 'creationdate': '2022-06-07T02:40:21+00:00', 'author': 'Charlotte Lorand', 'crossmarkdomains[1]': 'elsevier.com', 'crossmarkdomains[2]': 'sciencedirect.com', 'crossmarkdomainexclusive': 'true', 'crossmarkmajorversiondate': '2010-04-23', 'elsevierwebpdfspecifications': '7.0', 'keywords': 'Human-carnivore coexistence; Lethal control; Non-lethal management; Conservation interventions; Effectiveness; Evidence-based', 'moddate': '2022-06-07T02:40:21+00:00', 'subject': 'Science of the Total Environment, 838 (2022) 156195. doi:10.1016/j.scitotenv.2022.156195', 'title': "Effectiveness of interventions for managing human-large carnivore conflicts worldwide: Scare them off, don't remove them", 'doi': '10.1016/j.scitotenv.2022.1

In [15]:
rag_chain.invoke({"input": "We live in a suburban area and bears sometimes come into our town to eat from our fruit trees and trash. What are the best ways for us to prevent this as a community? We don’t want to have to get rid of our fruit trees…"})

{'input': 'We live in a suburban area and bears sometimes come into our town to eat from our fruit trees and trash. What are the best ways for us to prevent this as a community? We don’t want to have to get rid of our fruit trees…',
 'context': [Document(id='20f4ca4d-9fb3-42bd-b433-a4a0437bd86a', metadata={'producer': 'Acrobat Distiller 10.1.10 (Windows); modified using iText 4.2.0 by 1T3XT', 'creator': 'LaTeX with hyperref package', 'creationdate': '2020-01-16T12:33:42+05:30', 'keywords': '', 'moddate': '2025-05-27T12:12:25-07:00', 'subject': 'Conservation Biology 2020.34:232-243', 'wps-proclevel': '3', 'wps-journaldoi': '10.1111/(ISSN)1523-1739', 'author': '', 'title': 'Predicting intervention priorities for wildlife conflicts', 'wps-articledoi': '10.1111/cobi.13372', 'source': 'hwc/Baynham-Herd et al. 2019.pdf', 'total_pages': 12, 'page': 10, 'page_label': '242'}, page_content='242 Conﬂict Intervention Priorities\nhelps foster more effective collaboration (Game et al.\n2013; Lute et

In [16]:
prompt = "What cattle husbandry strategies might be helpful to prevent conflict if we live in wolf country?"

rag_chain.invoke({"input": prompt})

{'input': 'What cattle husbandry strategies might be helpful to prevent conflict if we live in wolf country?',
 'context': [Document(id='50b6a82e-13e1-4e8f-8a0a-5e7cd8a1ad50', metadata={'producer': 'PyPDF', 'creator': 'Elsevier', 'creationdate': '2022-06-07T02:40:21+00:00', 'author': 'Charlotte Lorand', 'crossmarkdomains[1]': 'elsevier.com', 'crossmarkdomains[2]': 'sciencedirect.com', 'crossmarkdomainexclusive': 'true', 'crossmarkmajorversiondate': '2010-04-23', 'elsevierwebpdfspecifications': '7.0', 'keywords': 'Human-carnivore coexistence; Lethal control; Non-lethal management; Conservation interventions; Effectiveness; Evidence-based', 'moddate': '2022-06-07T02:40:21+00:00', 'subject': 'Science of the Total Environment, 838 (2022) 156195. doi:10.1016/j.scitotenv.2022.156195', 'title': "Effectiveness of interventions for managing human-large carnivore conflicts worldwide: Scare them off, don't remove them", 'doi': '10.1016/j.scitotenv.2022.156195', 'robots': 'noindex', 'source': 'hwc